# Traditional Models

### Install All Required Libraries

In [1]:
!pip uninstall -y numpy scipy pandas statsmodels

Found existing installation: numpy 2.2.4
Uninstalling numpy-2.2.4:
  Successfully uninstalled numpy-2.2.4
Found existing installation: scipy 1.15.2
Uninstalling scipy-1.15.2:
  Successfully uninstalled scipy-1.15.2
Found existing installation: pandas 2.2.3
Uninstalling pandas-2.2.3:
  Successfully uninstalled pandas-2.2.3
Found existing installation: statsmodels 0.14.4
Uninstalling statsmodels-0.14.4:
  Successfully uninstalled statsmodels-0.14.4


In [2]:
!pip install numpy==1.24.4 scipy==1.10.1 pandas==1.5.3 statsmodels==0.13.5

  Using cached numpy-1.24.4.tar.gz (10.9 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [33 lines of output]
      Traceback (most recent call last):
        File "/opt/anaconda3/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 353, in <module>
          main()
        File "/opt/anaconda3/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/opt/anaconda3/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 112, in get_requires_for_build_wheel
          backend = _build_backend()
                    ^^^^^^^^^^^^^^^^
        File "/opt/anaconda3

In [7]:
!pip install pmdarima

  Using cached numpy-2.2.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (62 kB)
  Using cached pandas-2.2.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached scipy-1.15.2-cp312-cp312-macosx_12_0_arm64.whl.metadata (61 kB)
  Using cached statsmodels-0.14.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (9.2 kB)
Using cached numpy-2.2.4-cp312-cp312-macosx_11_0_arm64.whl (14.1 MB)
Using cached pandas-2.2.3-cp312-cp312-macosx_11_0_arm64.whl (11.4 MB)
Using cached scipy-1.15.2-cp312-cp312-macosx_12_0_arm64.whl (30.2 MB)
Using cached statsmodels-0.14.4-cp312-cp312-macosx_11_0_arm64.whl (9.9 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.4 which is incompatible.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.15.2 which is incompatible.
contourpy 1.2.0 requires numpy<2.0,>

In [8]:
# Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
import pmdarima as pm
import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'pmdarima'

In [ ]:
# Function to check stationarity using ADF Test
def test_stationarity(timeseries, label='Time Series'):
    result = adfuller(timeseries.dropna())
    print(f"ADF Test for {label}: p-value = {result[1]:.5f}")
    if result[1] < 0.05:
        print(f"{label} is Stationary.\n")
    else:
        print(f" {label} is NOT Stationary.\n")
    return result[1] < 0.05

In [ ]:
#  Function to apply ARMA model on differenced data
def fit_arma_model(ts, label='Dataset'):
    ts_diff = ts.diff().dropna()

    # Plot ACF and PACF to identify orders
    plot_acf(ts_diff)
    plt.title(f"ACF Plot - {label}")
    plt.show()

    plot_pacf(ts_diff)
    plt.title(f"PACF Plot - {label}")
    plt.show()

    # Fit ARMA(p=2, q=2) model manually
    arma_model = ARIMA(ts_diff, order=(2, 0, 2)).fit()
    print(f" ARMA(2,0,2) Summary for {label}:\n", arma_model.summary())

    # Forecast differenced values and revert to original scale
    future_diff = arma_model.forecast(steps=24)
    forecast_vals = ts.iloc[-1] + np.cumsum(future_diff)
    future_idx = pd.date_range(start=ts.index[-1], periods=25, freq='M')[1:]

    plt.figure(figsize=(10, 5))
    plt.plot(ts, label='Original')
    plt.plot(future_idx, forecast_vals, label='ARMA Forecast', color='green')
    plt.title(f"{label} - ARMA Forecast (24 Months)")
    plt.legend()
    plt.show()

In [ ]:
# Function to apply Auto ARIMA and forecast
def run_auto_arima(ts, label='Series'):
    arima_model = pm.auto_arima(ts, seasonal=False, trace=True,
                                stepwise=True, suppress_warnings=True)
    print(f"Auto ARIMA selected order for {label}: {arima_model.order}")

    # Fit final ARIMA model
    final_model = ARIMA(ts, order=arima_model.order).fit()
    prediction = final_model.forecast(steps=24)
    future_dates = pd.date_range(start=ts.index[-1], periods=25, freq='M')[1:]

    # Plot forecast
    plt.figure(figsize=(10, 5))
    plt.plot(ts, label='Historical')
    plt.plot(future_dates, prediction, label='Auto ARIMA Forecast', color='red')
    plt.title(f"{label} - Auto ARIMA Forecast (24 Months)")
    plt.legend()
    plt.show()

In [ ]:
# Load and Process jj.csv
jj_data = pd.read_csv("jj.csv")
jj_data.columns = ['Quarter', 'Sales']
jj_data['Quarter'] = pd.to_datetime(jj_data['Quarter'])
jj_data.set_index('Quarter', inplace=True)
jj_monthly = jj_data['Sales'].resample('M').ffill()

In [ ]:
# Load and Process AMZN.csv
amzn_data = pd.read_csv("AMZN.csv")
amzn_data['Date'] = pd.to_datetime(amzn_data['Date'])
amzn_data.set_index('Date', inplace=True)
amzn_monthly = amzn_data['Close'].resample('M').mean()

# Apply to JJ Dataset

In [ ]:
test_stationarity(jj_monthly.diff(), 'JJ Monthly Differenced')

In [ ]:
fit_arma_model(jj_monthly, label="Johnson & Johnson")

In [ ]:
run_auto_arima(jj_monthly, label="Johnson & Johnson")

# Apply to AMZN Dataset

In [9]:
test_stationarity(amzn_monthly.diff(), 'Amazon Monthly Differenced')

NameError: name 'test_stationarity' is not defined

In [10]:
fit_arma_model(amzn_monthly, label="Amazon")

NameError: name 'fit_arma_model' is not defined

In [ ]:
run_auto_arima(amzn_monthly, label="Amazon")

# Neural Network Models

In [ ]:
!pip install --upgrade numpy scipy pandas statsmodels

In [ ]:
# Imports for Deep Learning
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.random import set_seed
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Reproducibility
np.random.seed(42)
set_seed(42)

In [ ]:
# Preprocessing Function
def prepare_series(data, look_back=12):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i+look_back])
        y.append(data[i+look_back])
    return np.array(X), np.array(y)

In [ ]:
# Model Training & Forecasting
def build_and_forecast_nn(data, model_type='LSTM', label='Series', look_back=12):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data.values.reshape(-1, 1))

    X, y = prepare_series(scaled_data, look_back)
    X = X.reshape((X.shape[0], X.shape[1], 1))

    # Define the model
    model = Sequential()
    if model_type == 'LSTM':
        model.add(LSTM(64, activation='relu', input_shape=(look_back, 1)))
    elif model_type == 'GRU':
        model.add(GRU(64, activation='relu', input_shape=(look_back, 1)))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    # Train model
    model.fit(X, y, epochs=50, verbose=0)

    # Forecasting the next 24 months
    future_input = scaled_data[-look_back:].reshape(1, look_back, 1)
    future_preds = []
    for _ in range(24):
        pred = model.predict(future_input)[0][0]
        future_preds.append(pred)
        future_input = np.append(future_input[:, 1:, :], [[[pred]]], axis=1)

    # Rescale predictions
    forecast_vals = scaler.inverse_transform(np.array(future_preds).reshape(-1, 1)).flatten()

    # Plot forecast
    forecast_index = pd.date_range(start=data.index[-1], periods=25, freq='M')[1:]
    plt.figure(figsize=(10, 5))
    plt.plot(data.index, data.values, label='Historical Data')
    plt.plot(forecast_index, forecast_vals, label=f'{model_type} Forecast', color='purple' if model_type == 'LSTM' else 'orange')
    plt.title(f'{label} - {model_type} Forecast (24 Months)')
    plt.legend()
    plt.show()

    # Evaluation on train set
    y_pred_train = model.predict(X)
    y_true_train = y
    print(f" Evaluation Metrics for {label} using {model_type}:")
    print(f"- MSE: {mean_squared_error(y_true_train, y_pred_train):.5f}")
    print(f"- MAE: {mean_absolute_error(y_true_train, y_pred_train):.5f}\n")

## Run All Function

In [ ]:
# Load and Prepare jj.csv
jj_df = pd.read_csv("jj.csv")
jj_df.columns = ['Quarter', 'Sales']
jj_df['Quarter'] = pd.to_datetime(jj_df['Quarter'])
jj_df.set_index('Quarter', inplace=True)
jj_monthly = jj_df['Sales'].resample('M').ffill()

In [ ]:
# Load and Prepare AMZN.csv
amzn_df = pd.read_csv("AMZN.csv")
amzn_df['Date'] = pd.to_datetime(amzn_df['Date'])
amzn_df.set_index('Date', inplace=True)
amzn_monthly = amzn_df['Close'].resample('M').mean()

## Run LSTM and GRU for JJ Data

In [ ]:
print(" Johnson & Johnson Sales - LSTM Model")
build_and_forecast_nn(jj_monthly, model_type='LSTM', label='Johnson & Johnson Sales')

In [ ]:
print(" Johnson & Johnson Sales - GRU Model")
build_and_forecast_nn(jj_monthly, model_type='GRU', label='Johnson & Johnson Sales')

## Run LSTM and GRU for AMZN Data

In [ ]:
print(" Amazon Stock Price - LSTM Model")
build_and_forecast_nn(amzn_monthly, model_type='LSTM', label='Amazon Stock Price')

In [ ]:
print(" Amazon Stock Price - GRU Model")
build_and_forecast_nn(amzn_monthly, model_type='GRU', label='Amazon Stock Price')